In [1]:
import time
import datetime
import math
import numpy as np
import pandas as pd
import lightgbm as lgb
from dateutil.parser import parse
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error


train = pd.read_csv('./chouchoukan.csv')
dev = pd.read_csv('./chouchoukan_dev.csv')
test = pd.read_csv('./test_X.csv')
# train = train.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# test = test.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# print(train['holiday'])
del test['holiday']
del dev['holiday']
del train['holiday']
# train = train.astype('float64')
predictors = [f for f in train.columns if f not in ['phone_num','time_stamp']]

def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = math.sqrt(mean_squared_error(label,pred))
    return ('mse',score,False)

print('Training..')
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

# from sklearn.cross_validation import train_test_split  
# train_feat, val_feat, train_y, val_y = train_test_split(train[predictors],train['phone_num'],test_size = 0.2,random_state=42) 
train_feat = train[predictors]
train_y = train['phone_num']

val_feat = dev[predictors]
val_y = dev['phone_num']
lgb_train = lgb.Dataset(train_feat, train_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
lgb_val = lgb.Dataset(val_feat, val_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50000,
                valid_sets=lgb_val,
                verbose_eval=100,
                feval=evalerror,
                early_stopping_rounds=100)
print(pd.Series(gbm.feature_importance(),index = predictors).sort_values(ascending = False))
# import pickle
# with open('gbm_pm25_ld.pkl', 'wb') as f:
#     pickle.dump(gbm, f)
test_x = test[['loc_id','time_stamp']]


test_x['time_stamp'] = test_x['time_stamp'].apply(lambda x: x.split(':')[0])  


test_y = gbm.predict(test[predictors])




/home/zhangqq/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/zhangqq/.local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training..
Training until validation scores don't improve for 100 rounds.


/home/zhangqq/.local/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[100]	valid_0's rmse: 377.161	valid_0's mse: 377.161
[200]	valid_0's rmse: 226.379	valid_0's mse: 226.379
[300]	valid_0's rmse: 193.954	valid_0's mse: 193.954
[400]	valid_0's rmse: 188.485	valid_0's mse: 188.485
[500]	valid_0's rmse: 186.336	valid_0's mse: 186.336
[600]	valid_0's rmse: 185.302	valid_0's mse: 185.302
[700]	valid_0's rmse: 184.468	valid_0's mse: 184.468
[800]	valid_0's rmse: 183.849	valid_0's mse: 183.849
[900]	valid_0's rmse: 183.548	valid_0's mse: 183.548
[1000]	valid_0's rmse: 183.398	valid_0's mse: 183.398
Early stopping, best iteration is:
[996]	valid_0's rmse: 183.376	valid_0's mse: 183.376
loc_week                 5861
data_max_week_hour       5450
data_mean_week_hour      4598
hour_rate                3876
loc_rate                 3335
loc_hour                 2741
data_median_week_hour    2210
data_min_week_hour       2160
loc_weather              2142
hour_weather             2093
week_rate                2086
O3_8h                    2010
pressure             

/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
import time
import datetime
import math
import numpy as np
import pandas as pd
import lightgbm as lgb
from dateutil.parser import parse
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error


train = pd.read_csv('./chouchoukan.csv')
dev = pd.read_csv('./chouchoukan_dev.csv')
train = pd.concat([train, dev],axis = 0)

test = pd.read_csv('./test_X.csv')
# train = train.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# test = test.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# print(train['holiday'])
del test['holiday']

del train['holiday']
# train = train.astype('float64')
predictors = [f for f in train.columns if f not in ['phone_num','time_stamp']]

def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = math.sqrt(mean_squared_error(label,pred))
    return ('mse',score,False)

print('Training..')
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

# from sklearn.cross_validation import train_test_split  
# train_feat, val_feat, train_y, val_y = train_test_split(train[predictors],train['phone_num'],test_size = 0.2,random_state=42) 
train_feat = train[predictors]
train_y = train['phone_num']

# val_feat = dev[predictors]
# val_y = dev['phone_num']
lgb_train = lgb.Dataset(train_feat, train_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
# lgb_val = lgb.Dataset(val_feat, val_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=996,
                valid_sets=lgb_train,
                verbose_eval=100,
                feval=evalerror,
                )
print(pd.Series(gbm.feature_importance(),index = predictors).sort_values(ascending = False))
# import pickle
# with open('gbm_pm25_ld.pkl', 'wb') as f:
#     pickle.dump(gbm, f)
test_x = test[['loc_id','time_stamp']]


test_x['time_stamp'] = test_x['time_stamp'].apply(lambda x: x.split(':')[0])  


test_y = gbm.predict(test[predictors])




Training..


/home/zhangqq/.local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	training's rmse: 336.533	training's mse: 336.533
[200]	training's rmse: 184.953	training's mse: 184.953
[300]	training's rmse: 142.894	training's mse: 142.894
[400]	training's rmse: 126.167	training's mse: 126.167
[500]	training's rmse: 117.293	training's mse: 117.293
[600]	training's rmse: 111.598	training's mse: 111.598
[700]	training's rmse: 107.28	training's mse: 107.28
[800]	training's rmse: 103.76	training's mse: 103.76
[900]	training's rmse: 100.605	training's mse: 100.605
data_max_week_hour       6875
data_mean_week_hour      5319
loc_week                 5035
loc_hour                 3274
hour_rate                3228
loc_rate                 3174
week_rate                3141
data_min_week_hour       2358
hour_weather             2343
loc_weather              2245
data_median_week_hour    2051
week_weather             2023
pressure                 1709
O3_8h                    1571
hour_week                1384
PM10                     1378
week_num                 1374

/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
test_y = pd.DataFrame(test_y)
test_y.columns = ['num_of_people']
test_y['num_of_people'] = test_y['num_of_people'].astype('int32')
data = pd.concat([test_x, test_y],axis = 1)



data['num_of_people'][data['num_of_people']<=0] = 20
data.to_csv('submit.csv',index = False)

/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
train = pd.read_csv('./submit.csv')
dev = pd.read_csv('./submit248.csv')

score = math.sqrt(mean_squared_error(train['num_of_people'],dev['num_of_people']))
print(score)

163.8577013640076
